In [3]:
import os
import requests
from dotenv import load_dotenv
from langchain.tools import tool
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

# 1. 加载环境变量
load_dotenv()

AMAP_KEY = os.getenv("AMAP_KEY")

# 2. 定义查询天气的工具
@tool
def get_weather(city: str) -> str:
    """
    使用高德地图 Web API 查询中国城市的实时天气。
    参数:
        city: 城市名称（中文，如 "北京"、"深圳"）。
    返回:
        城市的实时天气描述，包括天气现象、温度、风力等。
    """
    url = f"https://restapi.amap.com/v3/weather/weatherInfo?key={AMAP_KEY}&city={city}&extensions=base&output=JSON"

    resp = requests.get(url).json()

    if resp.get("status") != "1":
        return f"无法查询到 {city} 的天气信息。错误信息: {resp}"

    lives = resp.get("lives", [])
    if not lives:
        return f"没有找到 {city} 的天气数据。"

    weather_info = lives[0]
    weather = weather_info["weather"]
    temperature = weather_info["temperature"]
    wind = weather_info["winddirection"]
    wind_power = weather_info["windpower"]
    humidity = weather_info["humidity"]
    report_time = weather_info["reporttime"]

    return (
        f"{city} 当前天气：{weather}，气温 {temperature}°C，"
        f"风向 {wind}，风力 {wind_power} 级，湿度 {humidity}%。"
        f"（数据更新时间：{report_time}）"
    )

# 3. 创建 LLM 与 Agent
model = ChatOpenAI(model="gpt-4o-mini")
agent = create_agent(
    model=model,
    tools=[get_weather],
    system_prompt="你是一名中国天气助手，用户提问时请自动调用工具获取准确的实时天气。"
)

# 4. 用户交互
result = agent.invoke({
    "messages": [
        {"role": "user", "content": "帮我查一下深圳的天气"}
    ]
})

print(result)
print(result["messages"][-1].content)


{'messages': [HumanMessage(content='帮我查一下深圳的天气', additional_kwargs={}, response_metadata={}, id='2476ebc5-5157-4dbc-a075-be1959ec2253'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 117, 'total_tokens': 132, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-CVdekBDgUVp6NLpkYD3OSwEjvZrTA', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--81b3d325-d9cf-4ad2-93c2-e8da5a26b6f3-0', tool_calls=[{'name': 'get_weather', 'args': {'city': '深圳'}, 'id': 'call_myFIgghZxiS1Rcof6KNP0evf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 117, 'output_tokens': 15, 'total_tokens': 132, 'input_token_details': {'audio': 0, 